### 출처
https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/

In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import LearningRateScheduler
from keras import regularizers

from sklearn.model_selection import train_test_split

import tensorflow as tf
import numpy as np
import os
import sys
from six.moves import cPickle
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.005
    if epoch > 100:
        lrate = 0.003
    return lrate

In [3]:
def load_data():
    # Modify path string. use your path which your dataset is in
    path =  r'C:\Users\strea\Links\baseline_code_final (2)'
    fpath = os.path.join(path, 'train_data')
    
    with open(fpath, 'rb') as f:
        d = cPickle.load(f, encoding='bytes')
    X_train = d['data']
    y_train = d['labels']
    X_train = X_train.reshape(X_train.shape[0], 3, 32, 32) # 3072개 숫자가 한줄로 이어진 배열 하나를 32x32짜리 배열 3개로 만듦
    X_train = X_train.transpose(0, 2, 3, 1)
    y_train = np.reshape(y_train, (len(y_train), 1))
    return X_train, y_train

x_train, y_train = load_data()

In [4]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=seed)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train[0][0][0:5]

array([[131., 136., 169.],
       [144., 150., 183.],
       [141., 148., 182.],
       [143., 150., 184.],
       [146., 156., 187.]], dtype=float32)

In [6]:
# z-score : 표준편차를 단위로서 보았을 때 측정치가 평균에서 얼마만큼 일탈하였는가를 보는 것이다. 
#           개인의 측정치 X와 평균치 M과의 차를 표준편차(SD)로 나눈 수이다.
mean = np.mean(x_train, axis=(0,1,2,3))
std = np.std(x_train, axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7) # 각 원소와 평균과의 차이를 표준편차로 나눔(ppt에 나옴 -> normalized data)
x_test = (x_test-mean)/(std+1e-7)
print(mean, std, x_train.shape, x_test.shape)
x_train[0][0][0:5]

120.98071 64.11213 (28000, 32, 32, 3) (12000, 32, 32, 3)


array([[0.15627757, 0.23426592, 0.7489891 ],
       [0.3590473 , 0.45263335, 0.96735656],
       [0.31225428, 0.421438  , 0.95175886],
       [0.34344962, 0.45263335, 0.9829542 ],
       [0.39024264, 0.54621935, 1.0297472 ]], dtype=float32)

In [7]:
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [8]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:])) 
# 32x32에 3x3짜리 필터로 padding은 사이즈 유지할 만큼 주고 / l2 regularization을 수행한다(오버피팅 방지), 그리고 input shape을 지정해준다
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2

In [9]:
# data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [14]:
# training
batch_size = 64
opt_rms = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
             optimizer=opt_rms,
             metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Instructions for updating:
Use tf.cast instead.
Epoch 1/125
437/437 [==============================] - 19s 44ms/step - loss: 2.1769 - acc: 0.3607 - val_loss: 1.8894 - val_acc: 0.4551
Epoch 2/125
437/437 [==============================] - 16s 36ms/step - loss: 1.5422 - acc: 0.5001 - val_loss: 1.3538 - val_acc: 0.5897
Epoch 3/125
437/437 [==============================] - 16s 36ms/step - loss: 1.3038 - acc: 0.5749 - val_loss: 1.2854 - val_acc: 0.6103
Epoch 4/125
437/437 [==============================] - 15s 34ms/step - loss: 1.1554 - acc: 0.6259 - val_loss: 1.2887 - val_acc: 0.6397
Epoch 5/125
437/437 [==============================] - 15s 34ms/step - loss: 1.0644 - acc: 0.6552 - val_loss: 1.0755 - val_acc: 0.6741
Epoch 6/125
437/437 [==============================] - 15s 35ms/step - loss: 1.0046 - acc: 0.6794 - val_loss: 1.1402 - val_acc: 0.6660
Epoch 7/125
437/437 [==============================] - 15s 35ms/step - loss: 0.9439 - acc: 0.7030 - val_loss: 0.8520 - val_acc: 0.7372
Epoch 8

437/437 [==============================] - 16s 36ms/step - loss: 0.5929 - acc: 0.8471 - val_loss: 0.6343 - val_acc: 0.8446
Epoch 60/125
437/437 [==============================] - 16s 37ms/step - loss: 0.5874 - acc: 0.8522 - val_loss: 0.6336 - val_acc: 0.8454
Epoch 61/125
437/437 [==============================] - 16s 37ms/step - loss: 0.5926 - acc: 0.8479 - val_loss: 0.6939 - val_acc: 0.8267
Epoch 62/125
437/437 [==============================] - 16s 37ms/step - loss: 0.5824 - acc: 0.8526 - val_loss: 0.6326 - val_acc: 0.8472
Epoch 63/125
437/437 [==============================] - 16s 37ms/step - loss: 0.5884 - acc: 0.8497 - val_loss: 0.5910 - val_acc: 0.8571
Epoch 64/125
437/437 [==============================] - 16s 37ms/step - loss: 0.5936 - acc: 0.8490 - val_loss: 0.6350 - val_acc: 0.8453
Epoch 65/125
437/437 [==============================] - 16s 37ms/step - loss: 0.5850 - acc: 0.8532 - val_loss: 0.6163 - val_acc: 0.8509
Epoch 66/125
437/437 [==============================] - 16s 3

437/437 [==============================] - 16s 36ms/step - loss: 0.8128 - acc: 0.8031 - val_loss: 0.8093 - val_acc: 0.8131
Epoch 120/125
437/437 [==============================] - 16s 36ms/step - loss: 0.8251 - acc: 0.8033 - val_loss: 0.7934 - val_acc: 0.8197
Epoch 121/125
437/437 [==============================] - 16s 36ms/step - loss: 0.8172 - acc: 0.8023 - val_loss: 0.8493 - val_acc: 0.8036
Epoch 122/125
437/437 [==============================] - 16s 36ms/step - loss: 0.8109 - acc: 0.8046 - val_loss: 0.7644 - val_acc: 0.8241
Epoch 123/125
437/437 [==============================] - 16s 36ms/step - loss: 0.8151 - acc: 0.8050 - val_loss: 0.9665 - val_acc: 0.7795
Epoch 124/125
437/437 [==============================] - 16s 37ms/step - loss: 0.8094 - acc: 0.8054 - val_loss: 0.7844 - val_acc: 0.8213
Epoch 125/125
437/437 [==============================] - 16s 37ms/step - loss: 0.8114 - acc: 0.8056 - val_loss: 0.8035 - val_acc: 0.8163


In [15]:
# save to disk
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = '60165060_trained_model.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\strea\Links\baseline_code_final (2)\saved_models\60165060_trained_model.h5 


In [17]:
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

12000/12000 [==============================] - 2s 137us/step

Test result: 81.625 loss: 0.803
